In [1]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.loader import DataLoader
import sys
import os
import torch
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)
from simulations.projectile_trajectory_2d import projectile_trajectory_2d
from gnn_model.graph_structure_from_trajecotry import node_data_list
from gnn_model.message_passing_MLP import GNN_MLP
from gnn_model.train_model import train_model

## Calling simulations to create data

In [2]:
# loading data
trajectory = projectile_trajectory_2d(30, 0.01, 35, 100)
graph_data_trajectory = node_data_list(trajectory, self_loop=True, complete_graph=False)

# initialising model
model = GNN_MLP(n_f=6, m_dim=2, out_channels=2, hidden_channels=128)
train_model(model, graph_data_trajectory, lr=0.01, epochs=100)

/Users/jacobtoot/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)


GNN_MLP()

In [4]:
# Output results
print("\nSimulation Results:")
for idx, data in enumerate(graph_data_trajectory):
    with torch.no_grad():
        pred_force = model(data.x, data.edge_index)  # Model predicts net force
        
        # Ground truth force
        ground_truth_force = data.y
        
        # Ensure correct shapes and conversion to NumPy for printing
        pred_force_np = pred_force.cpu().numpy().flatten()  # Convert to NumPy array and flatten
        ground_truth_np = ground_truth_force.cpu().numpy().flatten()  # Convert to NumPy and flatten

        # Print each component of the force vector
        print(f"Sample {idx + 1}:")
        for i in range(len(pred_force_np)):
            print(f"   Component {i}: Ground Truth = {ground_truth_np[i]:.6f}, Predicted = {pred_force_np[i]:.6f}")



Simulation Results:
Sample 1:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.324162
Sample 2:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.324065
Sample 3:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.323970
Sample 4:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.323874
Sample 5:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.323781
Sample 6:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.323688
Sample 7:
   Component 0: Ground Truth = 0.000000, Predicted = -0.000103
   Component 1: Ground Truth = -9.810000, Predicted = -9.323595
Sample 8:
   Compone